In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

from time import sleep, time
from threading import Thread
from scipy.cluster.hierarchy import fcluster
from scipy.stats import zscore

from data_processing import DataIterator
from testing import TestFactory, ClusteredInfo
from dtw import dtw as cur_dtw
from dtw_wrapper import DtwWrapper

In [2]:
def norm_1(x, y):
    return np.linalg.norm(x - y, ord=1)
def norm_2(x, y):
    return np.linalg.norm(x - y, ord=2)

# Rewriting data in our format

In [3]:
# data = pd.read_csv("../data/preprocessed_large.csv", header=None)

# indexes = pd.Index(range(0, 200))
# data_preprocessed = pd.DataFrame(columns=["obj", "ch", "label", *indexes])

# l = []
# for j in data.index:
#     df = pd.DataFrame({"obj": [j for i in range(3)], 
#                   "label": [data.loc[j, 0] for i in range(3)]},
#                  columns=["obj", "label", *indexes])
#     df.index.name = "ch"
#     df.loc[0, indexes] = data.loc[j, 1:200].values
#     df.loc[1, indexes] = data.loc[j, 201:400].values
#     df.loc[2, indexes] = data.loc[j, 401:600].values 
#     df.reset_index(inplace=True)
#     data_preprocessed = data_preprocessed.append(df, sort=False)
    
# data_preprocessed.reset_index(drop=True, inplace=True)


In [4]:
data = pd.read_csv("../data/preprocessed_3ch_50point_overlap0.csv", index_col=0)
it = DataIterator(data, 50, random_state=42)
tests = TestFactory(it=it)
x = tests.set_sample(2048)

In [ ]:
wrap = DtwWrapper(tests.X, hash(tests.infos), cur_dtw, norm_2, dtw_args={"z_normalize": True, "l": 0.2})
wrap.fill_distances()

dump
0
dump
30
dump
60
dump
90
dump
120
dump
150
dump
180
dump
210
dump
240
dump
270
dump
300
dump
330
dump
360
dump
390
dump
420
dump
450
dump
480
dump
510
dump
540
dump
570
dump
600
dump
630
dump
660
dump
690
dump
720
dump
750
dump
780
dump
810
dump
840
dump
870
dump
900
dump
930
dump
960
dump
990


In [ ]:
tmp = np.genfromtxt("../data/distances/dtwnorm_2z_normalizeTruel0.2-43544539162478229291.csv")
(tmp < 0).sum() / (tmp != -1000).sum()

In [ ]:
DtwWrapper(tests.X, hash(tests.infos), dtw, norm_2, dtw_args={"z_normalize": True, "l": 0.2})
info = tests.test_dtw(dtw, norm_2, dtw_args={"z_normalize": True, "l": 0.2})
info.cluster(24)
info.stats
index = info.stats.head(10).index.values
classifier_stat = {}
for i in info.stats.index:
    classifier_stat[i] = pd.Series(info.label[np.where(info.clusters_labels == i)[0]]).value_counts()
    
pd.DataFrame(classifier_stat).fillna(0)

In [6]:
1 % 4 == 1

True